## Build Functions for ELT

## Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 uninstall pandas pyarrow fireducks numpy -y
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppresses INFO and WARNING messages
import tensorflow as tf

from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

# Check if GPU is available
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

E0000 00:00:1733826811.208526   27407 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733826811.211676   27407 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num GPUs Available: 1


### Download, Transform, and Modeling All in One

In [13]:
f.predictions('tsla')

TSLA 5m Interval Timestamp: 2024-12-15 02:29:24 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.316279,0.342024,0.295918,0.215339
prob_up,0.119089,0.086855,0.187557,0.25
prob_static,0.702335,0.563149,0.614507,0.625
prob_down,0.178576,0.349996,0.197936,0.125
precision,0.511628,0.530017,0.497085,0.439528
recall,0.618474,0.620482,0.684739,0.598394
f1,0.56,0.571693,0.576014,0.506803
support,"[498.0, 498.0, 499.0]","[498.0, 498.0, 499.0]","[498.0, 498.0, 499.0]","[498.0, 498.0, 499.0]"


TSLA 15m Interval Timestamp: 2024-12-15 02:29:28 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.258416,0.307292,0.274775,0.201869
prob_up,0.362129,0.208748,0.334248,0.375
prob_static,0.475446,0.489333,0.427277,0.375
prob_down,0.162424,0.301919,0.238475,0.25
precision,0.470297,0.505208,0.481982,0.429907
recall,0.579268,0.591463,0.652439,0.560976
f1,0.519126,0.544944,0.554404,0.486772
support,"[166.0, 164.0, 165.0]","[166.0, 164.0, 165.0]","[166.0, 164.0, 165.0]","[166.0, 164.0, 165.0]"


TSLA 1h Interval Timestamp: 2024-12-15 02:29:30 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.013869,0.06124,0.081818,0.133113
prob_up,0.220304,0.309882,0.295231,0.25
prob_static,0.41994,0.378772,0.40776,0.625
prob_down,0.359756,0.311346,0.297009,0.125
precision,0.29562,0.329457,0.344156,0.380795
recall,0.3361,0.352697,0.439834,0.477178
f1,0.314563,0.340681,0.386157,0.423573
support,"[239.0, 241.0, 238.0]","[239.0, 241.0, 238.0]","[239.0, 241.0, 238.0]","[239.0, 241.0, 238.0]"


TSLA 1wk Interval Timestamp: 2024-12-15 02:29:36 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,static,up
kelly_1:2.5,0.091228,0.154717,0.004444,0.172727
prob_up,0.506859,0.40341,0.347016,0.5
prob_static,0.277503,0.34666,0.360794,0.375
prob_down,0.215638,0.24993,0.292191,0.125
precision,0.350877,0.396226,0.288889,0.409091
recall,0.408163,0.428571,0.270833,0.367347
f1,0.377358,0.411765,0.27957,0.387097
support,"[49.0, 48.0, 48.0]","[49.0, 48.0, 48.0]","[49.0, 48.0, 48.0]","[49.0, 48.0, 48.0]"


TSLA 1mo Interval Timestamp: 2024-12-15 02:29:37 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,static,up
kelly_1:2.5,-0.12,-0.05,0.02,0.1
prob_up,0.608605,0.580165,0.361627,0.5
prob_static,0.361284,0.258747,0.37976,0.25
prob_down,0.030111,0.161088,0.258613,0.25
precision,0.2,0.25,0.3,0.357143
recall,0.2,0.3,0.333333,0.5
f1,0.2,0.272727,0.315789,0.416667
support,"[10.0, 9.0, 10.0]","[10.0, 9.0, 10.0]","[10.0, 9.0, 10.0]","[10.0, 9.0, 10.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')